In [ ]:
%load_ext autoreload
%autoreload 2

import os,sys
import mne
import utils  #my code
import json
import matplotlib.pyplot as plt
import numpy as np
import h5py
import os
import matplotlib as mpl
import utils_tSNE as utsne

from matplotlib.widgets import Button
from matplotlib.text import Annotation
import globvars as gv
import utils_interactive_scatter as uis

data_dir = gv.data_dir

In [ ]:
rawname_ = 'S01_on_hold'  # no trem
rawname_ = 'S01_off_hold'
rawname_ = 'S01_off_move'
#rawname_ = 'S01_on_move'


#rawname_ = 'S02_off_hold'
#rawname_ = 'S02_on_hold'
rawnames = [rawname_]
rawnames = ['S01_off_hold', 'S01_off_move']
rn_str = ','.join(rawnames)

crop_time = -1
nraws_used_PCA = 2  # more precision

skip_feat = 32  #  80 ,128
subskip = 4
skip_tSNE = skip_feat * subskip

wsz = 256
#dim_PCA = ?
#effdim = ?
#n_feats_PCA = ?
n_channels = 7
skip_PCA = skip_feat

prefix = 'all'

In [ ]:
# regex = 
# utsne.findByPrefix(gv.data_dir, '', '', regex)

In [ ]:
#out_name_templ = '{}_{}tSNE_nrPCA{}_{}chs_nfeats{}_pcadim{}_skip{}_wsz{}_effdim{}_subskip{}'
regex = '{}_{}_tSNE_nrPCA{}_{}chs_nfeats([0-9]+)_pcadim([0-9]+)_skip{}_wsz{}_effdim([0-9]+)_subskip{}'.\
format(rn_str,prefix,nraws_used_PCA, n_channels, skip_PCA, wsz, subskip)

regex = '{}_{}_tSNE_nrPCA{}_.*_subskip{}'.\
format(rn_str,prefix,nraws_used_PCA, subskip)
#regex = '{}_.*tSNE.*'.format(rn_str)
if crop_time > 0:
    regex += '_crop{.1f}'.format(crop_time)
    
print(regex)

fnfound = utsne.findByPrefix(data_dir, rawname_, prefix, regex=regex)
assert len(fnfound) == 1, 'For {} found not single fnames {}'.format(rawname_,fnfound)
print(fnfound)

fname_tsne = fnfound[0]
fname_tsne_full = os.path.join( data_dir,fname_tsne) 
#S01_off_move_feats_7chs_nfeats609_skip32_wsz256.npz

In [ ]:
#os.stat( fname_feat_full )

In [ ]:
from datetime import datetime


In [ ]:
help(datetime.fromtimestamp)

In [ ]:
nfeats = 609
Xs = []
for rawname_ in rawnames:
    fname_feat = '{}_feats_7chs_nfeats{}_skip{}_wsz256.npz'.\
    format(rawname_, nfeats, skip_feat) 
    fname_feat_full = os.path.join(data_dir, fname_feat)
    #fname_tsne_full = os.path.join(data_dir, '{}_tsne_7chs_skip{}_wsz256.npz'.format(rawname_, skip_tSNE) )

    r = np.load(fname_feat_full, allow_pickle=1)
    mtime = os.stat( fname_feat_full ).st_mtime
    print(fname_feat)
    print( datetime.fromtimestamp(mtime), '(US format)' ) 

    X = r['X']
    Xs += [X]
X = np.vstack(Xs)
feature_names_all = r['feature_names_all']
#Xtimes = r['Xtimes']
#extdat = r['extdat']

skip = r['skip']
sfreq = r['sfreq']
windowsz = r['windowsz']
nedgeBins = r['nedgeBins']


###################################


r = np.load(fname_tsne_full, allow_pickle=1)
mtime = os.stat( fname_tsne_full ).st_mtime
print(fname_tsne)
print( datetime.fromtimestamp(mtime), '(US format)' ) 


tSNE_result = r['tSNE_result']

#colors = r['colors']
#markers = r['markers']
#legend_elements = r['legend_elements']
anns = r['anns']

print('len(X)={}, len(X)/subskip={}, tSNE[0][2].shape={}'.
      format(len(X), len(X)/subskip, tSNE_result[0][2].shape))

In [ ]:
Xtimes_almost_pri = []
PCA_files_pri = []
for rawname_ in rawnames:
    subj,medcond,task  = utils.getParamsFromRawname(rawname_)
    regex_nrPCA = '[0-9]+'
    regex_nfeats = '[0-9]+'
    regex_pcadim = '[0-9]+'
    regex = '{0}_{1}_PCA_nr({2})_[0-9]+chs_nfeats({3})_pcadim({4}).*'.\
        format(rawname_, prefix, regex_nrPCA, regex_nfeats, regex_pcadim)

    fnfound = utsne.findByPrefix(data_dir, rawname_, prefix, regex=regex)
    if len(fnfound) > 1:
        fnt = [0] * len(fnfound)
        for fni in range(len(fnt) ):
            fnfull = os.path.join(data_dir, fnfound[fni])
            fnt[fni] = os.path.getmtime(fnfull)
        fni_max = np.argmax(fnt)
        fnfound = [ fnfound[fni_max] ]
    assert len(fnfound) == 1, 'For {} found not single fnames {}'.format(rawname_,fnfound)
    fname_PCA = fnfound[0]
    fname_PCA_full = os.path.join( data_dir, fname_PCA )

    f = np.load(fname_PCA_full, allow_pickle=1)
    mtime = os.stat( fname_PCA_full ).st_mtime
    print(fname_PCA)
    print( datetime.fromtimestamp(mtime), '(US format)' ) 
    
    Xtimes_cur = f['Xtimes'] 
    Xtimes_almost_pri += [ Xtimes_cur ]
    PCA_files_pri += [f]


    print(Xtimes_cur.shape)


In [ ]:
# for better display
feature_names_all_mod = [name.replace('allf_','') for name in feature_names_all]
feature_names_all_mod = [name.replace('con_','') for name in feature_names_all_mod]

# select 30 preplexity

In [ ]:
alg_to_use = 'UMAP'
dtype_to_use = 'PCA'
param_to_use = 10
for tpl in tSNE_result:
    pi,si,X_embedded, seed, param, lrate, dtype, alg = tpl
    if param == param_to_use and dtype == dtype_to_use and alg == alg_to_use:
        break

In [ ]:
itypes = ['trem_L', 'notrem_L', 'hold_L', 'move_L']

In [ ]:
totskip = skip_tSNE

In [ ]:
windowsz

In [ ]:
totskip

In [ ]:
anns, anns_pri, Xtimes, dataset_bounds = utsne.concatAnns(rawnames, Xtimes_almost_pri)
ivalis = utils.ann2ivalDict(anns)
ivalis_tb, ivalis_tb_indarrays = utsne.getAnnBins(ivalis, Xtimes, nedgeBins, sfreq, 
                                                  totskip, windowsz, dataset_bounds)
ivalis_tb_indarrays_merged = utsne.mergeAnnBinArrays(ivalis_tb_indarrays)

print(Xtimes.shape, X.shape)

### markers
https://matplotlib.org/api/markers_api.html

In [ ]:
sind_str,medcond,task = utils.getParamsFromRawname(rawname_)
#mrk = ['<','>','o','^','v']
mrk = ['<','o','o','X','v']
#mrknames = ['_pres','_posts','','_pree','_poste']

In [ ]:
maintremside = gv.gp.gen_subj_info[subj]['tremor_side']

In [ ]:
mts_letter = maintremside[0].upper()

In [ ]:
anis = [ani for ani,ann in enumerate(anns) if ann['description'].find(mts_letter) >= 0]
anns_mts = anns[anis]

len(anns_mts)

print( 'Proportion of dataset covered by labels ',
      np.sum(anns_mts.duration) / Xtimes[-1] )

ivalis

# Prep colors

In [ ]:
its = [it for it in ivalis_tb_indarrays_merged if it.find(mts_letter) >= 0]
#anns_mts = anns[anis]

In [ ]:
ivalis_tb_indarrays_merged_mts = {}
for it in its:
    ivalis_tb_indarrays_merged_mts[it] = ivalis_tb_indarrays_merged[it]

In [ ]:
sum( [len(indarr) for indarr in ivalis_tb_indarrays_merged_mts.values()] ) / len(Xtimes[::4])

In [ ]:
Xtimes[::subskip].shape

In [ ]:
'fd'.startswith('ss')

In [ ]:
ivalis['notrem_L'][0]

In [ ]:
ivalis_tb_indarrays['notrem_L'][0]

In [ ]:
len(ivalis_tb_indarrays_merged_mts['trem_L'])

In [ ]:
print(anns_mts)
anns_mts.onset

In [ ]:
colors, markers =utsne.prepColorsMarkers(mts_letter, anns, Xtimes[::subskip],
           nedgeBins, windowsz, sfreq, totskip, mrk,gv.gp.mrknames, 
                                         gv.gp.color_per_int_type,
                                        dataset_bounds = dataset_bounds)

legend_elements = utsne.prepareLegendElements(mrk,
                                              gv.gp.mrknames,
                                              gv.gp.color_per_int_type, 
                                              task, skipExt = 0 )

In [ ]:
nonneut = np.array(colors) != gv.gp.neutcolor
num_nonneut = sum(  nonneut)

In [ ]:
print( 'Proportion of dataset non-neut colors ',
      num_nonneut / len(colors) )

In [ ]:
np.where(nonneut)[0][0] / len(colors)

In [ ]:
np.where(nonneut)[0][-1] / len(colors)

# Read muscle info

In [ ]:
import utils_preproc as upre
maintremside = 'left'

EMG_per_hand = {'right':['EMG061_old', 'EMG062_old'], 'left':['EMG063_old', 'EMG064_old' ] }
chnames_emg = EMG_per_hand[maintremside]

In [ ]:
emg_rectconvs = []
emg_rectconvs2 = []
true_times_pri = []
dataset_bounds_uncut_bins = []
shift = 0
for rawname_ in rawnames:
    rectconv_fname_full = os.path.join(data_dir, '{}_emg_rectconv.fif'.format(rawname_) )
    print(rectconv_fname_full)
    emg_rectconv_cur = mne.io.read_raw_fif(rectconv_fname_full)
    emg_rectconvs += [emg_rectconv_cur.copy()]
    emg_rectconvs2 += [emg_rectconv_cur.copy()]
    true_times_pri += [emg_rectconv_cur.times]
    
    st,end = 0, len(emg_rectconv_cur.times)
    st += shift; end += shift
    dataset_bounds_uncut_bins += [(st,end)]
    
#emg_rectconv = mne.concatenate_raws(emg_rectconvs2)
emg_rectconv = upre.concatRaws(emg_rectconvs2)
tbs,tbe = emg_rectconv.time_as_index([0,300])
rectconv_emg, ts_ = emg_rectconv[chnames_emg, tbs:tbe]

anns_naive = utsne.concatAnnsNaive(rawnames, true_times_pri)

emg_rectconv.set_annotations(anns_naive)
emg_rectconv_ps = utils.getEMGperHand(emg_rectconv)



In [ ]:
dataset_bounds_uncut_bins

In [ ]:
#emg_rectconvs[0].plot(duration=50,scalings='auto');

# emg_rectconv.plot(duration=50,scalings='auto');

# emg_rectconvs[0].plot(duration=50,scalings='auto');

# emg_rectconvs[1].plot(duration=50,scalings='auto');

# emg_rectconvs[0].annotations

# help(emg_rectconvs[0].apply_function)

# emg_rectconvs[0].apply_function()

# emg_rectconvs[0].annotations.onset

# emg_rectconvs[0].annotations.duration

# emg_rectconvs[0].set_annotations( mne.Annotations([],[],[]) )



In [ ]:
%matplotlib qt

In [ ]:
emg_rectconv_ps[maintremside].plot(duration=20, scalings='auto');

In [ ]:
# tasks = []
# ivalis_pri = []
# anns_pri = []

# ivalis_tb = {}  # only bounds
# ivalis_tb_indarrays = {}  # entire arrays
# globend = Xtimes[-1] + nedgeBins/sfreq

# tshift = 0
# assert len(rawnames) == 1 # otherwise I need to load both rawnames, better to load anns from run_tSNE
# for rawname_ in rawnames:
#     subj,medcond,task  = utils.getParamsFromRawname(rawname_)
#     tasks += [task]

#     anns_fn = rawname_ + '_anns.txt'
#     anns_fn_full = os.path.join(data_dir, anns_fn)
#     anns = mne.read_annotations(anns_fn_full)
#     #raw.set_annotations(anns)
    
#     utils.ann2ivalDict(anns)

#     ivalis_cur = utils.ann2ivalDict(anns)
#     for it in ivalis_cur:
#         intervals = ivalis_cur[it]
#         intervals_bins = []
#         ivalis_tb_indarrays[it] = []
#         for st,end,it_ in intervals:
#             bnd0 = min(len(Xtimes)-1, st * sfreq // totskip   + windowsz // totskip -1   )
#             bnd1 = min(len(Xtimes)-1, end *sfreq // totskip    + windowsz // totskip -1   )
#             if bnd1 - bnd0 > 0:
#                 intervals_bins += [(bnd0,bnd1,it_)]
#                 ivalis_tb_indarrays[it] += [np.arange(bnd0,bnd1, dtype=int)]
#         ivalis_tb[it] =  intervals_bins
    
    


# resinds = {}
# for an in anns:
#     for descr in annot_colors_cur:
#         if an['description'] != descr:
#             continue
#         col = annot_colors_cur[descr]

#         start = an['onset']
#         end = start + an['duration']

#         timesBnds, indsBnd, sliceNames = getIntervalSurround( start,end, extend,
#                                                             times=Xtimes)
#         #print('indBnds in color prep ',indsBnd)
#         for ii in range(len(indsBnd)-1 ):
#             # do not set prestart, poststart for left recording edge
#             if start <= nedgeBins/sfreq and ii in [0,1]:
#                 continue
#             # do not set preend, posted for right recording edge
#             #globend = Xtimes_almost[-1] + nedgeBins/sfreq
#             globend = Xtimes[-1] + nedgeBins/sfreq
#             if  globend - end <= nedgeBins/sfreq and ii in [3,4]:
#                 continue
#             # window size correction because it goes _before_
#             bnd0 = min(len(Xtimes)-1, indsBnd[ii]   + windowsz // totskip -1   )
#             bnd1 = min(len(Xtimes)-1, indsBnd[ii+1] + windowsz // totskip -1   )
#             #inds2 = slice( bnd0, bnd1 )
#             inds2 = range( bnd0, bnd1 )

#             #inds2 = slice( indsBnd[ii], indsBnd[ii+1] )
#             #markers[inds2] = mrk[ii]

#             for jj in inds2:
#                 colors [jj] = col
#                 markers[jj] = mrk[ii]

## just test that we imported info correctly

In [ ]:
#help(ax.scatter)

## Emph some points

In [ ]:
# create and add an annotation object (a text label)
def annotate(axis, text, x, y):
    text_annotation = Annotation(text, xy=(x, y), xycoords='data')
    axis.add_artist(text_annotation)

In [ ]:
#ivalis_tb_indarrays['trem_L'][0]

In [ ]:
def emph_interval(ax, int_name,int_ind, ls='--',msz=5,msz_emph=22,
                 lines_only=True):
    # prepare highlight of some interval
    emph_inds = ivalis_tb_indarrays[int_name][int_ind]

    alpha_noemph = 0.5

    sizes = np.array( [msz] * len(colors) )
    sizes[emph_inds] = msz_emph

    colors_rgba = utsne.colNames2Rgba(colors)
    colors_rgba = np.vstack(colors_rgba)
    noemph_inds = np.setdiff1d(np.arange(len(colors), ), emph_inds)
    colors_rgba[noemph_inds,-1] = alpha_noemph

#%matplotlib qt
#%matplotlib inline
#ax = plt.gca()

    if not lines_only:
        utsne.plotMultiMarker(ax,X_embedded[:,0], X_embedded[:,1], c = colors_rgba, s=sizes,
                              m=markers);
    # connect by line
    ax.plot(X_embedded[emph_inds,0], X_embedded[emph_inds,1], lw=1, ls=ls)
    


In [ ]:
plt.close('all')

In [ ]:
#%debug
ax = plt.gca()
emph_interval(ax,'trem_L',1, lines_only=0)
ax.legend(handles=legend_elements, loc='upper right')

In [ ]:
ivalis['trem_L']

## Scale channels to similar size

In [ ]:
dat_emg_scaled_ = rectconv_emg.copy()
dat_emg_scaled_ -= np.mean(dat_emg_scaled_, axis=1)[:,None]
dat_emg_scaled_ /= (np.quantile(dat_emg_scaled_,0.95,axis=1) - np.quantile(dat_emg_scaled_,0.05,axis=1))[:,None]

In [ ]:
mn = np.quantile(dat_emg_scaled_, 0.05); 
mx = np.quantile(dat_emg_scaled_,0.95); 

In [ ]:
emg_skip = 5
ts_skipped = emg_rectconv.times[::emg_skip]
dat_emg_scaled = dat_emg_scaled_[:,::emg_skip]

In [ ]:
plt.plot(dat_emg_scaled.T)

In [ ]:
plt.plot(dat_emg_scaled.T[:3 * 256//emg_skip])

In [ ]:
skip_feat

In [ ]:
nedgeBins

In [ ]:
dataset_bounds_uncut_bins

In [ ]:
def plotEMG(ax, ptind, offset = 3):
    '''
    offset is in sec
    ptind -- index of feature bin
    ''' 
    ax.cla()
    dbbs = [(dbs// skip_feat,dbe// skip_feat) for (dbs,dbe) in dataset_bounds_uncut_bins ]
    ptind_hole = utils.indHoleToNoHole(ptind, dbbs, 
                                            nedgeBins // skip_feat)
    #ptind_sec = (ptind * skip_tSNE ) /  sfreq
    #ptind_sec = (ptind_feat_hole * skip_feat ) /  sfreq
    ptind_sec = (ptind * skip_feat ) /  sfreq
    print('ptind sec ', ptind_sec)
    t0,t1 = max(emg_rectconv.times[0],ptind_sec - offset), \
    min(ptind_sec + offset, emg_rectconv.times[-1] )
    assert t0 <= t1, (t0,t1)
    tb0,tb1 = emg_rectconv.time_as_index([t0,t1])
    tb0_skip = tb0 // emg_skip
    tb1_skip = tb1 // emg_skip
    
    times_toplot = ts_skipped[tb0_skip:tb1_skip]
    
    assert len(rectconv_emg) == 2

    ax.set_title('EMG recmav time={:.2f}'.format(Xtimes[ptind] ) )
    for ii in range(2):
        #print('Lalalal')
        dat = dat_emg_scaled[ii,tb0_skip:tb1_skip]
        print(dat.shape)
        ax.plot( times_toplot, dat , label = chnames_emg[ii])
    
    #print((ptind[0] * skip) / sfreq)
    ax.axvline(x= (ptind * skip_tSNE) / sfreq, c='r', ls=':', lw=5)
    ax.set_xlabel('time, [s]')

    ax.set_ylim(mn,mx)
    ax.set_xlim(t0,t1)
    ax.legend(loc='lower left')
    ax.fill_between([ptind_sec-windowsz/sfreq, ptind_sec], 
                    [mn,mn], [mx,mx], color='r',alpha=0.2)
    
ax = plt.gca()
ptind = int( ivalis['trem_L'][0][1] * sfreq / skip_feat )
plotEMG(ax, ptind, 10)
#plotEMG(ax, ptind, 5)
#plotEMG(ax, ptind)


In [ ]:
ax = plt.gca()
plotEMG(ax, (40*256)//skip_tSNE)

In [ ]:
#help(ax.fill_between)

In [ ]:
#300 * 45  /256

In [ ]:
#X_embedded

# Plot interactive scatter

In [ ]:
%matplotlib qt

In [ ]:
feature_names_all_mod

In [ ]:
help(ax.remove)

In [ ]:
#global gxlabels_were_set
#gxlabels_were_set = 0
def plotPtInfo(axEMG,axFeat,trueInd, show_labels='strong', qstrong=0.95):
    '''
    trueInd -- index of feature bin
    '''
    if axEMG is not None:
        plotEMG(axEMG,trueInd)
    
    if axFeat is not None:
        dat = X[trueInd * (skip_tSNE//skip_feat)]
        
        axFeat.cla()
        axFeat.set_title('Feat time={:.2f}'.format(Xtimes[trueInd] ) )
        axFeat.plot(dat)
        
        dd = np.abs(dat )
        q = np.quantile(dd,qstrong)
        strongInds = np.where( dd  > q ) [0]
        strongestInd = np.argmax(dd)
        ax.axhline(y=q, c='purple', ls=':')
        ax.axhline(y=-q, c='purple', ls=':')
        
        #global gxlabels_were_set
        #if not gxlabels_were_set:
        if show_labels != 'none':
            axFeat.set_xticks(range(len(feature_names_all)))
            if show_labels == 'all':
                axFeat.set_xticklabels(feature_names_all_mod,rotation=90)
            if show_labels == 'strong':
                tl = np.array( feature_names_all_mod[:] )
                inds_ = set(np.arange(len(feature_names_all))) - set(strongInds)
                inds_ = list(inds_)
                tl[ inds_ ] = ''
                axFeat.set_xticklabels(tl,rotation=90)


            tls = axFeat.get_xticklabels()
            if len(tls):
                for i in strongInds:
                    tls[i].set_color("red")
                tls[strongestInd].set_color("blue")
                
        axFeat.set_xlim(0, len(feature_names_all)-1 )
        axFeat.set_ylim(-1.5,1.5)
        axFeat.axhline(y=0, c='r', ls=':')

        
#         if show_labels == 'strong':
#             axFeat.set_xticks(range(len(strongInds)))
#             axFeat.set_xticklabels(np.array(feature_names_all_mod)[strongInds],rotation=90)
            
#             tls = axFeat.get_xticklabels()
#             if len(tls):
#                 si = list(strongInds).index(strongestInd)
#                 tls[si].set_color("blue")

%matplotlib qt

nr = 1; nc=2
assert nr > 0,  'Nothing was selected'
ww = 15; hh = 3
fig,axs = plt.subplots(nrows=nr, ncols=nc, figsize= (nc*ww,nr*hh), sharex='none'
                       , gridspec_kw={'width_ratios': [1, 3]})
axs = axs.reshape((nr,nc))
#%debug
for i,trueInd in enumerate([10]): #  [:1]):
    axEMG = axs[i,0]
    axFeat = axs[i,1]
    plotPtInfo(axEMG,axFeat,trueInd,qstrong=0.99)#, show_labels='none')
#     if i != len(selected) -1:
#         axFeat.set_xticklabels(['']*len(feature_names_all_mod))

plt.tight_layout()

In [ ]:
%matplotlib qt

In [ ]:
#markers == '^'

In [ ]:
for i in range(len(colors)):
    if colors[i] == 'g' and markers[i] in ['^', 'v', '>', '<']:
        markers[i] = 'o'

## Finally plot

In [ ]:
len(X_embedded)

In [ ]:
len(colors)

In [ ]:
len(markers)

In [ ]:
show_feat_info = 1
show_emg_info = 1
show_time_as_ptlabel = 1
show_all_pts_under_cursor = 0
ind_offset = 5    # how many neighbors to mark

fig,axs = plt.subplots(nrows=3, ncols=1, figsize=(20, 25), gridspec_kw={'height_ratios': [5, 1, 1]})
plt.subplots_adjust(bottom=0.22,top=0.99,left=0.03,right=0.99)
axScat = axs[0]; axEMG = axs[1]; axFeat = axs[2]

if show_emg_info:
    a1 = axEMG 
else: 
    a1 = None

if show_feat_info:
    a2 = axFeat
else:
    a2 = None

msz_normal = 25
#msz_normal = 40

custom_marker_size = {'^':msz_normal*3, '<':msz_normal*3}
    
def draw_scatterplot():
    # since I I use multiple marker types, I have to make several separate scatter plots
    # it gives separates "artists" which makes indexing more confusing, 
    # so I have to save the outcome for future use
    r = utsne.plotMultiMarker(axScat,X_embedded[:,0], X_embedded[:,1], 
                              c = colors, s=msz_normal,
                      m=markers, picker=True, 
                              custom_marker_size = custom_marker_size)
    axScat.legend(handles=legend_elements, loc='upper left')
    return r

# draw the initial scatterplot
global markerset,resinds,scs
markerset,resinds,scs = draw_scatterplot()

global selected, selected_all_info
selected = []; selected_all_info = []

# define the behaviour -> what happens when you pick a dot on the scatterplot by clicking close to it
def onpick(event):
    global selected, resinds, scs, selected_all_info
    offset = 0  # offset in putting labels
    ptinds = event.ind
    
    trueInds = []
    for ptind in ptinds:
        artistInd = scs.index(event.artist)
        trueInds += [ resinds[artistInd] [ptind] ] 

    selected_all_info += [(event, resinds, ptinds, scs)]
    # step 2: save the actual coordinates of the click, so we can position the text label properly
    click_pos_x = event.mouseevent.xdata
    click_pos_y = event.mouseevent.ydata
    
    trueInd = trueInds[0]
    #selected += trueInds
    selected += [trueInd]
    
    x,y = X_embedded[trueInd,:]
    xerr = abs(x - click_pos_x)
    yerr = abs(y - click_pos_y)
    #print('{}(true={}): errs=({:.2f},{:.2f})'.format(ptind,trueInd, xerr,yerr) )
    
    print('true ind={}: X_emb=({:.2f},{:.2f})'.
          format(trueInd,x,y) )

    plotPtInfo(a1,a2,trueInd)

    if show_all_pts_under_cursor:
        inds_pts_to_surrond = trueInds
    else:
        inds_pts_to_surrond = [trueInd]
    # if the dots are to close one to another, a list of dots clicked is returned by the matplotlib library
    for i in inds_pts_to_surrond:
        if ind_offset == 0:
            inds_range = [i]
        else:
            inds_range = range( max(0,i-ind_offset), min(i+ind_offset, (X_embedded.shape[0])-1  ) )
        
        for ind_cur in inds_range:
            time_cur = Xtimes[ind_cur]
            
            if show_time_as_ptlabel:
                lbl_content = time_cur
            else:
                lbl_content = ind_cur
                
            if ind_cur == i:
                #label = '!!{}'.format(label0)
                label = '!!{:.1f}'.format(lbl_content)
            else:
                label = '{:.1f}'.format(lbl_content)
                #label = '{}->{}'.format(i,ind_cur)#.format(label0, generated_labels[ind_cur] )

            x,y = X_embedded[ind_cur,:]
            annotate(axScat,label, x + offset, y + offset)

        # step 6: force re-draw
        axScat.figure.canvas.draw_idle()

fig.canvas.mpl_connect('pick_event', onpick)

def onclick_clearall(event):
    # step 1: we clear all artist object of the scatter plot
    for ax in axs:
        ax.cla()

    global markerset,resinds,scs, selected
    markerset,resinds,scs = draw_scatterplot()
    ax.figure.canvas.draw_idle()
    selected = []

bt_width = 0.1
bt_height = 0.025
# create the "clear all" button, and place it somewhere on the screen
ax_clear_all = plt.axes([0.0, 0.0, 0.1, bt_height])
button_clear_all = Button(ax_clear_all, 'Clear all')
# link the event handler function to the click event on the button
button_clear_all.on_clicked(onclick_clearall)


int_type = 'trem_L' 
vshift = bt_height/10
nints = len(ivalis_tb_indarrays[int_type] )
# funs = [0]*nints
# for int_ind in range(nints ):
#     emph_interval(axScat,int_type,int_ind, lines_only=1)
    
#     ax_trem_show = plt.axes([1.0-bt_width, (bt_height + vshift) *int_ind ,
#                              bt_width, bt_height])
#     #int_ind = 0
#     button_trem_cur = Button(ax_trem_show, '{}_{}'.format(int_type,int_ind))
#     onclick_bttrem = lambda event: emph_interval(axScat,int_type,int_ind, 
#                                                  msz_emph=msz_normal*2)
#     funs[int_ind] = onclick_bttrem
#     #fun = lambda x: print('ffd')
#     # def onclick_bttrem(event):
#     #     emph_interval(axScat,int_type,int_ind)
#     #     print('fdfd')
#     button_trem_cur.on_clicked(funs[int_ind])
from matplotlib.widgets import TextBox
ax_trem_show = plt.axes([1.0-bt_width-3e-3, (bt_height + vshift) *0 + 1e-3,
                              bt_width, bt_height])
text_box = TextBox(ax_trem_show, '{} ind (< {})'.format(int_type,nints), initial='')
def submit(text): 
    emph_interval(axScat,int_type,int(text), msz_emph=msz_normal*2)
    plt.draw()
text_box.on_submit(submit)
    

plt.plot()
print("scatterplot done")
plt.show()

In [ ]:
ivalis['trem_L']

In [ ]:
selected

In [ ]:
(event, resinds, ptinds,scs) = selected_all_info[0]

In [ ]:
event.__dict__

In [ ]:
print(scs.index(event.artist))
scs

In [ ]:
print(len(X_embedded),len(X))

In [ ]:
resinds

In [ ]:
# for ptind in ptinds:
#     artistInd = scs.index(event.artist)
#     trueInds += [ resinds[artistInd] [ptind] ] 

# selected_all_info += [(event, resinds, ptinds)]

In [ ]:
%matplotlib qt

nr = len(selected); nc=2
assert nr > 0,  'Nothing was selected'
ww = 15; hh = 3
fig,axs = plt.subplots(nrows=nr, ncols=nc, figsize= (nc*ww,nr*hh), sharex='none'
                       , gridspec_kw={'width_ratios': [1, 3]})
axs = axs.reshape((nr,nc))
#%debug
for i,trueInd in enumerate(selected): #  [:1]):
    axEMG = axs[i,0]
    axFeat = axs[i,1]
    plotPtInfo(axEMG,axFeat,trueInd,qstrong=0.99)#, show_labels='none')
    if i != len(selected) -1:
        axFeat.set_xticklabels(['']*len(feature_names_all_mod))

plt.tight_layout()

In [ ]:
skip_tSNE

In [ ]:
skip_feat